<a href="https://colab.research.google.com/github/HeywoodChan/video-classification-3d-cnn-pytorch/blob/master/model_garden_openai_api_llama3_2_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Get started with Llama 3.2 models

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_openai_api_llama3_2.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_openai_api_llama3_2.ipynb"">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_openai_api_llama3_2.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_openai_api_llama3_2.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

## Overview

This notebook demonstrates how to get started with using the OpenAI library and demonstrates how to leverage multimodal capabilities of Llama 3.2 models as Model-as-service (MaaS).

### Objective

- Configure OpenAI SDK for the Llama 3.2 Completions API
- Chat with Llama 3.2 models with different prompts and model parameters
- Build and use Llama 3.2 GenAI powered application for Car Damage Assessment.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [4]:
! pip3 install --upgrade --quiet google-cloud-aiplatform openai gradio

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [5]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [3]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [4]:
PROJECT_ID = "upheld-quanta-442214-i7"  # @param {type:"string"}

# Only `us-central1` is supported region for Llama 3.2 models using Model-as-a-Service (MaaS).
LOCATION = "us-central1"

### Create a Cloud Storage bucket

Create a storage bucket to store tutorial artifacts.

In [5]:
BUCKET_NAME = "cuhk_fashion_project_2024_batch_1"  # @param {type:"string"}

BUCKET_URI = f"gs://{BUCKET_NAME}"

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [6]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://cuhk_fashion_project_2024_batch_1/...
ServiceException: 409 A Cloud Storage bucket named 'cuhk_fashion_project_2024_batch_1' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Initialize Vertex AI SDK for Python

In [7]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import libraries

Import libraries to use in this tutorial.

In [8]:
import json
import re
import uuid
from io import BytesIO

import gradio as gr
import matplotlib.pyplot as plt
# Chat completions API
import openai
from google.auth import default, transport
from google.cloud import storage
from PIL import Image

### Helpers functions

In [9]:
def visualize_image_from_bucket(bucket_name: str, blob_name: str) -> None:
    """Visualizes an image stored in a Google Cloud Storage bucket."""
    try:
        # Create a client for interacting with Google Cloud Storage
        storage_client = storage.Client()

        # Get a reference to the bucket and blob
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(blob_name)

        # Download the image data into memory
        image_data = blob.download_as_bytes()

        # Open the image using PIL
        image = Image.open(BytesIO(image_data))

        # Display the image using matplotlib
        plt.figure(figsize=(10, 10))  # Set the figure size (adjust as needed)
        plt.imshow(image)
        plt.axis("off")  # Turn off axis labels
        plt.show()

    except Exception as e:
        print(f"Error visualizing image: {e}")

### Configure OpenAI SDK for the Llama 3.2 Chat Completions API

To configure the OpenAI SDK for the Llama 3.2 Chat Completions API, you need to request the access token and initialize the client pointing to the Llama 3.2 endpoint.


#### Authentication

You can request an access token from the default credentials for the current environment. Note that the access token lives for [1 hour by default](https://cloud.google.com/docs/authentication/token-types#at-lifetime); after expiration, it must be refreshed.


In [10]:
credentials, _ = default()
auth_request = transport.requests.Request()
credentials.refresh(auth_request)

Then configure the OpenAI SDK to point to the Llama 3.2 Chat Completions API endpoint.

Notice, only `us-central1` is supported region for Llama 3.2 models using Model-as-a-Service (MaaS).

In [11]:
MODEL_LOCATION = "us-central1"
MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

client = openai.OpenAI(
    base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
    api_key=credentials.token,
)

#### Llama 3.2 models

You can experiment with various supported Llama 3.2 models.

This tutorial use Llama 3.2 90B Vision Instruct using Model-as-a-Service (MaaS). Using Model-as-a-Service (MaaS), you can access Llama 3.2 models in just a few clicks without any setup or infrastructure hassles.

You can also access Llama models for self-service in Vertex AI Model Garden, allowing you to choose your preferred infrastructure. [Check out Llama 3.2 model card](https://console.cloud.google.com/vertex-ai/publishers/meta/model-garden/llama3-2?_ga=2.31261500.2048242469.1721714335-1107467625.1721655511) to learn how to deploy a Llama 3.2 models on Vertex AI.

In [12]:
MODEL_ID = "meta/llama-3.2-90b-vision-instruct-maas"  # @param {type:"string"} ["meta/llama-3.2-90b-vision-instruct-maas"]

### Chat with Llama 3.2

Use the Chat Completions API to send a multi-model request to the Llama 3.2 model.

#### Hello, Llama 3.2

In [13]:
def generate_chat_response(image_path, max_tokens=4096):
    # Ensure max_tokens is defined
    max_tokens = 4096

    # Call the API to generate a chat response
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "image_url": {
                            "url": image_path
                        },
                        "type": "image_url",
                    },
                    {"text": """
Background:
I am conducting a data science master's project focused on analyzing 10,000 fashion images to extract detailed attributes for each item. Your task is to meticulously analyze each image and identify specific attributes from the provided categories. The output must adhere strictly to the JSON format illustrated in the example below. Ensure high accuracy and consistency in your classifications to facilitate downstream data analysis.

________________________________________
Instructions:
1. Category Selection:
   For each fashion item image, identify and classify attributes based on the following categories. Each category is uniquely numbered and precisely defined to ensure comprehensive and granular data collection.

   • **Category 0: Fashion Main Category**
       - **Description:** Broad primary categories encompassing all types of fashion items.
       - **Examples:** Clothing, Footwear, Accessories, Outerwear, Bags, Jewelry, Hats.

   • **Category 1: Fashion Item Sub Category**
       - **Description:** Specific subcategories under each main category for precise classification.
       - **Examples:** Blazer, Jeans, T-Shirt, Sneakers, Handbag, Necklace, Fedora.

   • **Category 2: Patterns**
       - **Description:** The arrangement of forms, lines, or shapes on the surface of the fashion item, contributing to its aesthetic appeal.
       - **Examples:** Stripes, Polka Dots, Chevron, Houndstooth, Paisley, Geometric, Abstract, None.

   • **Category 3: Prints**
       - **Description:** Repeated decorative designs or motifs applied to the fabric or surface of the fashion item.
       - **Examples:** Floral Print, Animal Print, Camouflage Print, Graphic Prints, Artistic Prints, Tie-Dye, Digital Prints, None.

   • **Category 4: Graphics**
       - **Description:** Illustrations, logos, text, or other visual elements that are part of the fashion item's design.
       - **Examples:** Logo Embroidery, Graphic Tees, Statement Text, Brand Symbols, Decorative Illustrations, Minimalist Graphics, None.

   • **Category 5: Materials, Fabrics, and Finishes**
       - **Description:** Specifies the materials, fabric compositions, and finishes used in the item.
       - **Examples:** 100% Merino Wool, Modal Fiber Blend, Recycled Polyester, Brushed Chenille, Organic Cotton, Denim, Leather, Satin.

   • **Category 6: Fit**
       - **Description:** Describes how the clothing item fits the body, indicating the intended silhouette.
       - **Examples:** Fitted, Relaxed, Oversized, Tailored, Slim, Regular, Loose.

   • **Category 7: Silhouette**
       - **Description:** Refers to the overall shape or outline of the garment, contributing to its style and appearance.
       - **Examples:** Slim, Boxy, A-line, Straight, Flared, Peplum, Structured.

   • **Category 8: Length**
       - **Description:** Indicates the length of the clothing item, providing information on its coverage.
       - **Examples:** Mini, Knee-length, Midi, Ankle, Maxi, Cropped, Floor-length.

   • **Category 9: Design Features**
       - **Description:** Highlights specific design elements and structural features of the garment.
       - **Examples:** Double-breasted, Notched Collar, Asymmetric Hem, Patch Pockets, Zipper Closures, Embroidery, Sequins, Ruffles, Buttons, Belted Waist.

   • **Category 10: Embellishments**
       - **Description:** Details any additional decorative elements that enhance the item's aesthetic.
       - **Examples:** Embroidery, Sequins, Patches, Appliqués, Beading, Rhinestones, Lace.

   • **Category 11: Style**
       - **Description:** Refers to the distinctive appearance or design characteristic of the fashion item, aligned with specific fashion movements or personal aesthetics.
       - **Examples:** Classic, Minimalist, Bohemian, Streetwear, High-Fashion, Eclectic, Sporty, Preppy.

   • **Category 12: Theme**
       - **Description:** The underlying subject or motif that gives direction to the design of the fashion item.
       - **Examples:** Nautical, Military, Vintage, Futuristic, Tropical, Gothic, Romantic, Urban.

   • **Category 13: Overall Inspiration**
       - **Description:** The primary influence or source of creativity behind the fashion item's design, reflecting cultural, artistic, or temporal inspirations.
       - **Examples:** Audrey Hepburn-inspired, Americana, Retro, Avant-garde, Sustainable, Eclectic, Bohemian, Futuristic.

   • **Category 14: Main Color**
       - **Description:** The dominant color of the fashion item, serving as the primary visual identifier.
       - **Examples:** White, Black, Red, Blue, Green, Yellow, Pink, Purple, Neutral Tones, Pastels, Metallics, Pantone 11-0602 TCX (White), Pantone 14-0852 TCX (Spectra Yellow).

   • **Category 15: Sub Color**
       - **Description:** Secondary or accent colors present in the item that complement or contrast with the main color.
       - **Examples:** Off-white, Cream, Light Beige, Ivory, Neon Green, Metallic Silver, Navy Blue, Dusty Rose, Burnt Orange, Sky Blue, Teal, Burgundy.

   • **Category 16: Usage Categories**
       - **Description:** Contexts or occasions for which the item is intended, indicating its suitability for specific events or activities.
       - **Examples:** Formal Wear, Casual, Business Casual, Athletic, Evening Wear, Streetwear, Resort Wear, Wedding, Office, Party, Travel, Outdoor, Beachwear.

   • **Category 17: Gender-Based Categories**
       - **Description:** Specifies the intended gender for the fashion item, reflecting design and sizing considerations.
       - **Examples:** Male, Female, Non-binary, Gender Fluid, Unisex, Androgynous.

   • **Category 18: Seasonality**
       - **Description:** Specifies the season(s) the item is designed for, indicating appropriate materials and designs for weather conditions.
       - **Examples:** Spring, Summer, Fall, Winter, All-Season, Transitional.

   • **Category 19: Embellishments and Accessories**
       - **Description:** Details any additional decorative elements or attached accessories that enhance functionality or style.
       - **Examples:** Belts, Hoods, Scarves, Patches, Appliqués, Buckles, Chains, Pompoms.

   • **Category 20: Closure Type**
       - **Description:** Describes how the garment is fastened, affecting both functionality and style.
       - **Examples:** Buttons, Zipper, Hook and Loop, Tie, Snap, Buckle, Magnetic Closures, Velcro.

   • **Category 21: Target Age Group**
       - **Description:** Specifies the intended age demographic for the fashion item, guiding design and marketing strategies.
       - **Examples:** Teens (13-19), Young Adults (20-30), Adults (31-50), Seniors (51+), Children (Under 13), All Ages, Toddlers (1-3).

   • **Category 22: Popularity and Sales Status**
       - **Description:** Indicates whether the fashion item is currently popular or a hot-selling item in 2023, reflecting market trends and consumer demand.
       - **Examples:**
           - **Highly Popular:** Widely recognized and sought after, frequently featured in fashion media.
           - **Trending:** Gaining popularity rapidly, often seen in influencer posts and social media platforms.
           - **Moderately Popular:** Known within certain demographics or regions, maintaining steady sales.
           - **Not Popular:** Limited recognition and low sales figures.
           - **Unknown:** Popularity cannot be determined from the available information.
   • **Category 23: Description the characteristic of input clothes or shoes or Accessories in 200 words.**
       - **Description:** Description the characteristic of input clothes or shoes or Accessories in 200 words.
       - **Examples:**
        **The image displays a women's navy blue midi dress, embellished with delicate white floral embroidery, set against a clean white background. The dress is styled with thin spaghetti straps that lead to a sweetheart neckline, enhancing its feminine appeal. The embroidery, composed of small, stylized flowers and leaves, is dispersed across the dress in a seemingly random yet balanced pattern. The white floral details stand out prominently against the dark navy background, creating a striking contrast. The dress falls below the knee, indicating a midi length, which offers a sophisticated and adaptable style. The fabric appears lightweight and potentially slightly sheer, suggesting a comfortable and flowy fit. The overall impression is one of elegant simplicity and delicate craftsmanship. The monochromatic color scheme of navy and white adds to the dress's timeless and versatile nature. The uncluttered white background keeps the focus solely on the dress, showcasing its design and details effectively. The embroidery likely involves a combination of thread and possibly small beads or sequins, lending a subtle texture or shimmer to the floral motifs.

2. Attention to Detail:
   - Examine every aspect of the fashion item, ensuring that no attribute is overlooked.
   - Consider elements such as stitching, texture, embellishment placement, and unique design aspects that may not be immediately apparent.
   - If an attribute spans multiple categories (e.g., a patterned fabric with a specific closure type), ensure each category is accurately populated.

3. Consistency and Accuracy:
   - Maintain uniformity in category responses to facilitate reliable data aggregation.
   - Avoid ambiguous or subjective terms; choose the most precise term from the provided examples.
   - In cases where multiple applicable examples exist, prioritize based on prominence and visibility in the image.

4. Handling Uncertainty:
   - If an attribute is not discernible from the image, default to "Unknown" rather than "N/A" unless **Category 0** explicitly dictates otherwise.
   - Ensure that "Unknown" is used judiciously to reflect genuine uncertainty without compromising data integrity.

#### 2. Output Format:
• Your response should be **only** a JSON object covering Categories 0 to 23.
• If **Category 0** is selected as "Other", "Not Applicable", or "N/A", set all Categories 1 to 23 to "N/A".
• ***Must Follow the JSON example*** and output format below **precisely**. Do **not** include any additional text or explanations beyond the JSON structure.
• ***Must Follow the JSON example*** and output format **precisely**. Do **not** include any additional text or explanations beyond the JSON structure.
• ***Must Follow the JSON example*** and output format **precisely**. Do **not** include any additional text or explanations beyond the JSON structure.
• JSON output must include key "Category 0: Fashion Main Category", and Value "Clothing" and Comma
• Need to inference result from Category 0 to Category 23. Do not put "Unknown" to Category 0 to Category 23


#### 3. JSON Example:
```json
{
  "Category 0: Fashion Main Category": "Clothing",
  "Category 1: Fashion Item Sub Category": "Blazer",
  "Category 2: Patterns": "Stripes",
  "Category 3: Prints": "Graphic Prints",
  "Category 4: Graphics": "Logo Embroidery",
  "Category 5: Materials, Fabrics, and Finishes": "100% Merino Wool",
  "Category 6: Fit": "Fitted",
  "Category 7: Silhouette": "Tailored",
  "Category 8: Length": "Ankle-length",
  "Category 9: Design Features": "Double-breasted, Notched Collar, Asymmetric Hem",
  "Category 10: Embellishments": "Sequins",
  "Category 11: Style": "Classic",
  "Category 12: Theme": "Americana",
  "Category 13: Overall Inspiration": "Audrey Hepburn-inspired",
  "Category 14: Main Color": "White, Pantone 11-0602 TCX",
  "Category 15: Sub Color": "Off-white, Cream",
  "Category 16: Usage Categories": "Formal Wear",
  "Category 17: Gender-Based Categories": "Unisex",
  "Category 18: Seasonality": "Winter",
  "Category 19: Embellishments and Accessories": "None",
  "Category 20: Closure Type": "Buttons",
  "Category 21: Target Age Group": "Young Adults (20-30)",
  "Category 22: Popularity and Sales Status": "Trending",
  "Category 23: Description the characteristic of input clothes or shoes or Accessories in 200 words Status": "The image displays a women's navy blue midi dress, embellished with delicate white floral embroidery, set against a clean white background. The dress is styled with thin spaghetti straps that lead to a sweetheart neckline, enhancing its feminine appeal. The embroidery, composed of small, stylized flowers and leaves, is dispersed across the dress in a seemingly random yet balanced pattern. The white floral details stand out prominently against the dark navy background, creating a striking contrast. The dress falls below the knee, indicating a midi length, which offers a sophisticated and adaptable style. The fabric appears lightweight and potentially slightly sheer, suggesting a comfortable and flowy fit. The overall impression is one of elegant simplicity and delicate craftsmanship. The monochromatic color scheme of navy and white adds to the dress's timeless and versatile nature. The uncluttered white background keeps the focus solely on the dress, showcasing its design and details effectively. The embroidery likely involves a combination of thread and possibly small beads or sequins, lending a subtle texture or shimmer to the floral motifs."

Please DO NOT output Json Format as below.
**Category 23: Description the characteristic of input clothes or shoes or Accessories in 200 words Status**
The image displays a women's gray skirt, set against a clean white background. The skirt is styled with a pleated design, enhancing its feminine appeal. The pleats are evenly spaced and fall just above the knee, indicating a midi length, which offers a sophisticated and adaptable style. The fabric appears lightweight and potentially slightly sheer, suggesting a comfortable and flowy fit. The overall impression is one of elegant simplicity and delicate craftsmanship. The monochromatic color scheme of gray adds to the skirt's timeless and versatile nature. The uncluttered white background keeps the focus solely on the skirt, showcasing its design and details effectively.
Error parsing JSON from response for image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_10.jpg: Expecting value: line 1 column 1 (char 0)
Image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_10.jpg resulted in an empty or invalid row. Retrying...
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_10.jpg (Attempt 3/3)
**Category 0: Fashion Main Category**
Clothing

  **Category 1: Fashion Item Sub Category**
  Skirt

  **Category 2: Patterns**
  None

  **Category 3: Prints**
  None

  **Category 4: Graphics**
  None

  **Category 5: Materials, Fabrics, and Finishes**
  Unknown

  **Category 6: Fit**
  Unknown

  **Category 7: Silhouette**
  Unknown

  **Category 8: Length**
  Unknown

  **Category 9: Design Features**
  Unknown

  **Category 10: Embellishments**
  Unknown

  **Category 11: Style**
  Unknown

  **Category 12: Theme**
  Unknown

  **Category 13: Overall Inspiration**
  Unknown

  **Category 14: Main Color**
  Unknown

  **Category 15: Sub Color**
  Unknown

  **Category 16: Usage Categories**
  Unknown

  **Category 17: Gender-Based Categories**
  Unknown

  **Category 18: Seasonality**
  Unknown

  **Category 19: Embellishments and Accessories**
  Unknown

  **Category 20: Closure Type**
  Unknown

  **Category 21: Target Age Group**
  Unknown

  **Category 22: Popularity and Sales Status**
  Unknown

Please DO NOT output Json Format as below.
  **Answer:**

  **Category 0: Fashion Main Category**
  Clothing

  **Category 1: Fashion Item Sub Category**
  Blazer

  **Category 2: Patterns**
  None

  **Category 3: Prints**
  None

  **Category 4: Graphics**
  None

  **Category 5: Materials, Fabrics, and Finishes**
  100% Merino Wool

  **Category 6: Fit**
  Fitted

  **Category 7: Silhouette**
  Tailored

  **Category 8: Length**
  Ankle-length

  **Category 9: Design Features**
  Double-breasted, Notched Collar, Asymmetric Hem

  **Category 10: Embellishments**
  Sequins

  **Category 11: Style**
  Classic

  **Category 12: Theme**
  Americana

  **Category 13: Overall Inspiration**
  Audrey Hepburn-inspired

  **Category 14: Main Color**
  White, Pantone 11-0602 TCX

  **Category 15: Sub Color**
  Off-white, Cream

  **Category 16: Usage Categories**
  Formal Wear

  **Category 17: Gender-Based Categories**
  Unisex

  **Category 18: Seasonality**
  Winter

  **Category 19: Embellishments and Accessories**
  None

  **Category 20: Closure Type**
  Buttons

  **Category 21: Target Age Group**
  Young Adults (20-30)

  **Category 22: Popularity and Sales Status**
  Trending

  **Category 23: Description the characteristic of input clothes or shoes or Accessories in 200 words Status**
  The image displays a women's navy blue midi dress, embellished with delicate white floral embroidery, set against a clean white background. The dress is styled with thin spaghetti straps that lead to a sweetheart neckline, enhancing its feminine appeal. The embroidery, composed of small, stylized flowers and leaves, is dispersed across the dress in a seemingly random yet balanced pattern. The white floral details stand out prominently against the dark navy background, creating a striking contrast. The dress falls below the knee, indicating a midi length, which offers a sophisticated and adaptable style. The fabric appears lightweight and potentially slightly sheer, suggesting a comfortable and flowy fit. The overall impression is one of elegant simplicity and delicate craftsmanship. The monochromatic color scheme of navy and white adds to the dress's timeless and versatile nature. The uncluttered white background keeps the focus solely on the dress, showcasing its design and details effectively. The embroidery likely involves a combination of thread and possibly small beads or sequins, lending a subtle texture or shimmer to the floral motifs.
  Error parsing JSON from response for image gs://cuhk_fashion_report_2024_image_batch1/Elywood_1.jpg: Expecting value: line 1 column 1 (char 0)
  Image gs://cuhk_fashion_report_2024_image_batch1/Elywood_1.jpg resulted in an empty or invalid row. Retrying...
  Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_1.jpg (Attempt 3/3)



**Do NOT put "Unknown" or "None"** to Category 0 to Category 23 as Jason Format below.

  "Category 2: Patterns": "None",
  "Category 3: Prints": "None",
  "Category 4: Graphics": "None",
  "Category 5: Materials, Fabrics, and Finishes": "Unknown",
  "Category 6: Fit": "Unknown",
  "Category 7: Silhouette": "Unknown",
  "Category 8: Length": "Unknown",
  "Category 9: Design Features": "Unknown",
  "Category 10: Embellishments": "Unknown",
  "Category 11: Style": "Unknown",
  "Category 12: Theme": "Unknown",
  "Category 13: Overall Inspiration": "Unknown",
  "Category 14: Main Color": "White",
  "Category 15: Sub Color": "Gold",
  "Category 16: Usage Categories": "Unknown",
  "Category 17: Gender-Based Categories": "Unknown",
  "Category 18: Seasonality": "Unknown",
  "Category 19: Embellishments and Accessories": "None",
  "Category 20: Closure Type": "Buckle",
  "Category 21: Target Age Group": "Unknown",
  "Category 22: Popularity and Sales Status": "Unknown",

}
""", "type": "text"},
                ],
            },
        ],
        max_tokens=max_tokens,
    )

    # Print the response
    #print(response.choices[0].message.content)

    # Return the response content
    return response.choices[0].message.content

In [14]:
import json
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font

# Function to process a single response and append it to the Excel file
def process_and_append_response(image_path, response_content, output_excel="/content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx"):

    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

    """
    Processes the API response, extracts JSON content, and appends it to an Excel file.
    The first column will include the image path as a clickable hyperlink.

    Args:
        image_path (str): Path to the input image.
        response_content (str): Response content from the API.
        output_excel (str): Path to save the Excel file.
    Returns:
        bool: True if the row was successfully appended, False otherwise.
    """
    try:
        # Extract the JSON part from the response content
        json_start = response_content.find("{")
        json_end = response_content.rfind("}") + 1
        json_string = response_content[json_start:json_end]
        json_data = json.loads(json_string)  # Parse JSON content
    except Exception as e:
        print(f"Error parsing JSON from response for image {image_path}: {e}")
        return False

    # Prepare a dictionary for the DataFrame row
    row_data = {"image_path": image_path}
    for key, value in json_data.items():
        # Use only the category description (after ":")
        column_name = key.split(":")[-1].strip()
        row_data[column_name] = value

    # Convert row to a DataFrame
    df_row = pd.DataFrame([row_data])

    # Append the row to the Excel file incrementally
    try:
        with pd.ExcelWriter(output_excel, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
            if 'Sheet1' in writer.sheets:
                startrow = writer.sheets['Sheet1'].max_row
            else:
                startrow = 0
            df_row.to_excel(writer, index=False, header=writer.sheets == {}, startrow=startrow)

        print(f"Result for image {image_path} appended to {output_excel}")
        return True
    except Exception as e:
        print(f"Error writing to Excel for image {image_path}: {e}")
        return False

# Function to check if all columns for an image are N/A in the Excel file
def is_row_empty(image_path, output_excel="fashion_data.xlsx"):
    """
    Checks if all columns are N/A for a given image in the Excel file.

    Args:
        image_path (str): Path to the input image.
        output_excel (str): Path to the Excel file.
    Returns:
        bool: True if all columns are N/A for the given image, False otherwise.
    """
    try:
        # Load the Excel file
        df = pd.read_excel(output_excel)

        # Find the row corresponding to the image_path
        row = df[df["image_path"] == image_path]
        if row.empty:
            return True  # Row doesn't exist; consider it empty

        # Check if all columns (except 'image_path') are N/A
        return row.iloc[0, 1:].isna().all()
    except FileNotFoundError:
        print(f"Excel file {output_excel} not found. Treating as empty.")
        return True  # File doesn't exist; consider it empty
    except Exception as e:
        print(f"Error checking row for image {image_path}: {e}")
        return True



# Main logic to process multiple images with retry mechanism
def process_images_with_retry(image_paths, token_limit, output_excel="/content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx"):
    """
    Processes a list of images, retries up to 3 times if all columns are N/A, and saves results to Excel.

    Args:
        image_paths (list): List of image paths to process.
        token_limit (int): Token limit for the API.
        output_excel (str): Path to save the Excel file.
    """
    for image_path in image_paths:
        retry_count = 0
        max_retries = 3

        while retry_count < max_retries:
            try:
                print(f"Processing image: {image_path} (Attempt {retry_count + 1}/{max_retries})")

                # Call the API to get the response content
                response_content = generate_chat_response(image_path, token_limit)

                # Process the response and append it to the Excel file
                success = process_and_append_response(image_path, response_content, output_excel)

                # Check if the row is valid (not all N/A)
                if success and not is_row_empty(image_path, output_excel):
                    print(f"Image {image_path} successfully processed and saved.")
                    break  # Exit the retry loop if successful
                else:
                    print(f"Image {image_path} resulted in an empty or invalid row. Retrying...")
                    retry_count += 1
            except Exception as e:
                print(f"Unexpected error processing {image_path}: {e}")
                retry_count += 1

        if retry_count == max_retries:
            print(f"Failed to process image {image_path} after {max_retries} attempts. Skipping to next image.")

# Example usage for multiple images
#image_paths = [
#     #"gs://cuhk_fashion_report_2024_image_batch1/Dazzle_10.jpg"
#     "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_1.jpg",
#     #"gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_2.jpg",
#     #"gs://cuhk_fashion_report_2024_image_batch1/Dazzle_10.jpg"
#]

# Call the main function
#process_images_with_retry(image_paths, 4096)

In [15]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_1.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_10.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_100.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_101.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_102.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_103.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_104.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_105.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_106.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_107.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_108.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_109.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_11.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_110.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_111.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_112.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_113.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_114.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_115.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_116.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_117.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_118.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_119.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_12.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_120.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_121.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_122.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_123.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_124.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_125.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_126.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_127.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_128.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_129.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_13.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_130.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_131.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_132.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_133.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_134.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_135.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_136.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_137.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_138.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_139.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_14.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_140.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_141.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_142.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_143.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_144.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_145.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_146.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_147.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_148.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_149.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_15.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_150.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_16.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_17.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_18.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_19.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_2.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_20.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_21.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_22.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_23.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_24.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_25.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_26.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_27.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_28.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_29.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_3.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_30.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_31.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_32.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_33.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_34.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_35.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_36.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_37.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_38.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_39.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_4.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_40.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_41.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_42.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_43.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_44.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_45.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_46.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_47.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_48.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_49.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_5.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_50.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_51.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_52.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_53.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_54.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_55.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_56.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_57.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_58.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_59.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_6.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_60.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_61.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_62.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_63.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_64.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_65.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_66.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_67.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_68.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_69.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_7.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_70.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_71.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_72.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_73.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_74.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_75.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_76.jpg",


          "gs://cuhk_fashion_report_2024_image_batch1/Edition_77.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_78.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_79.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_8.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_80.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_81.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_82.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_83.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_84.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_85.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_86.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_87.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_88.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_89.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_9.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_90.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_91.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_92.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_93.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_94.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_95.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_96.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_97.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_98.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_99.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
credentials, _ = default()
auth_request = transport.requests.Request()
credentials.refresh(auth_request)

MODEL_LOCATION = "us-central1"
MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

client = openai.OpenAI(
      base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
      api_key=credentials.token,
    )


# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_77.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_78.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_79.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_8.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_80.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_81.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_82.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_83.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_84.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_85.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_86.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_87.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_88.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_89.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_9.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_90.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_91.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_92.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_93.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_94.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_95.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_96.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_97.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_98.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Edition_99.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_77.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_77.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Edition_77.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Edition_77.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_78.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_78.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Edition_78.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Edition_78.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_79.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_79.jpg (Attempt 1/3)
Result

In [1]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

NameError: name 'default' is not defined

In [2]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_1.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_10.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_100.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_101.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_102.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_103.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_104.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_105.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_106.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_107.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_108.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_109.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_11.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_110.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_111.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_112.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_113.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_114.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_115.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_116.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_117.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_118.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_119.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_12.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_120.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_121.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_122.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_123.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_124.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_125.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_126.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_127.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_128.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_129.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_13.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_130.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_131.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_132.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_133.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_134.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_135.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_136.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_137.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_1.jpg
Error processing image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_1.jpg: name 'process_images_with_retry' is not defined. Skipping to next image.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_10.jpg
Error processing image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_10.jpg: name 'process_images_with_retry' is not defined. Skipping to next image.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_100.jpg
Error processing image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_100.jpg: name 'process_images_with_retry' is not defined. Skipping to next image.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_101.jpg
Error processing image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_101.jpg: name 'process_images_with_retry' is not defined. Skipping to next image.
Processing image: gs://cuhk_fashio

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_138.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_139.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_14.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_140.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_141.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_142.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_143.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_144.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_145.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_146.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_147.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_148.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_149.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_15.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_150.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_16.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_17.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_18.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_19.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_2.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_20.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_21.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_22.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_23.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_24.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_25.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_26.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_27.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_28.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_29.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_3.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_30.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_31.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_32.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_33.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_34.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_35.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_36.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_37.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_38.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_39.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_4.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_40.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_41.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_42.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_43.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_44.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_45.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_46.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_47.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_48.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_49.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_5.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_50.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_51.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_52.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_53.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_54.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_55.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_56.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_57.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_58.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_59.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_6.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_60.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_61.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_138.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_138.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_138.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_138.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_139.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_139.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_139.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_139.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_14.jpg
Processing image: gs://cuhk_fashion_report_

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_61.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_62.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_63.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_64.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_65.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_66.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_67.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_68.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_69.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_7.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_70.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_71.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_72.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_73.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_74.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_75.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_76.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_77.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_78.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_79.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_8.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_80.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_81.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_82.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_83.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_84.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_85.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_86.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_87.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_88.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_89.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_9.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_90.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_91.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_92.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_93.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_94.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_95.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_96.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_97.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_98.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_99.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_61.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_61.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_61.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_61.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_62.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_62.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_62.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_62.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_63.jpg
Processing image: gs://cuhk_fashion_report_2024_ima

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_1.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_10.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_100.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_101.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_102.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_103.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_104.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_105.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_106.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_107.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_108.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_109.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_11.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_110.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_111.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_112.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_113.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_114.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_115.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_116.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_117.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_118.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_119.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_12.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_120.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_121.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_122.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_123.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_124.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_125.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_126.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_127.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_128.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_129.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_13.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_130.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_131.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_132.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_133.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_1.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_1.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Elywood_1.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Elywood_1.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_10.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_10.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Elywood_10.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Elywood_10.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_100.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_100.jpg (Attempt 1/3)
Result f

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_134.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_135.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_136.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_137.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_138.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_139.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_14.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_140.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_141.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_142.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_143.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_144.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_145.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_146.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_147.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_148.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_149.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_15.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_150.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_151.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_152.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_153.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_154.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_155.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_156.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_157.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_158.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_159.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_16.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_160.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_17.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_18.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_19.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_2.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_20.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_21.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_22.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_23.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_24.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_25.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_26.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_27.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_28.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_29.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_3.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_30.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_31.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_32.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_33.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_34.jpg",

]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_134.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_134.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Elywood_134.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Elywood_134.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_135.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_135.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Elywood_135.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Elywood_135.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_136.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_136.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_35.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_36.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_37.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_38.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_39.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_4.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_40.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_41.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_42.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_43.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_44.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_45.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_46.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_47.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_48.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_49.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_5.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_50.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_51.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_52.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_53.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_54.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_55.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_56.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_57.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_58.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_59.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_6.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_60.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_61.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_62.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_63.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_64.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_65.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_66.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_67.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_68.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_69.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_7.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_70.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_71.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_72.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_73.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_74.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_75.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_76.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_77.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_78.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_79.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_8.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_80.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_81.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_82.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_83.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_84.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_85.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_86.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_87.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_88.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_89.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_9.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_90.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_91.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_92.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_93.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_94.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_95.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_96.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_97.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_98.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Elywood_99.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_35.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_35.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Elywood_35.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Elywood_35.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_36.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_36.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Elywood_36.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Elywood_36.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_37.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Elywood_37.jpg (Attempt 1/3)
Result

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1113_15431c34-ec86-40b2-8234-4f192c871119_1170x.png",
          "gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1114_5b3555bc-ea37-43e5-9069-fb11029f30b2_1880x.png",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_1.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_10.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_100.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_101.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_102.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_103.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_104.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_105.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_106.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_107.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_108.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_109.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_11.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_110.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_111.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_112.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_113.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_114.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_115.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_116.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_117.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_118.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_119.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_12.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_120.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_121.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_122.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_123.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_124.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_125.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_126.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_127.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_128.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_129.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_13.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_130.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_131.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_132.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_133.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_134.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_135.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_136.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_137.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_138.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_139.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_14.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_140.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_141.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_142.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_143.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_144.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_145.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_146.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_147.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_148.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_15.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_16.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_17.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_18.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_19.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_2.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_20.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_21.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_22.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_23.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_24.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_25.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_26.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_27.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_28.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_29.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_3.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_30.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_31.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_32.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_33.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_34.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_35.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_36.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_37.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_38.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_39.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_4.jpg",

]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1113_15431c34-ec86-40b2-8234-4f192c871119_1170x.png
Processing image: gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1113_15431c34-ec86-40b2-8234-4f192c871119_1170x.png (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1113_15431c34-ec86-40b2-8234-4f192c871119_1170x.png appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1113_15431c34-ec86-40b2-8234-4f192c871119_1170x.png successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1114_5b3555bc-ea37-43e5-9069-fb11029f30b2_1880x.png
Processing image: gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1114_5b3555bc-ea37-43e5-9069-fb11029f30b2_1880x.png (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1114_5b3555bc-ea37-43e5-9069-fb11029f30b2_1880x.png ap

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [

          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_40.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_41.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_42.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_43.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_44.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_45.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_46.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_47.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_48.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_49.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_5.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_50.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_51.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_52.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_53.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_54.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_55.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_56.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_57.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_58.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_59.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_6.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_60.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_61.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_62.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_63.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_64.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_65.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_66.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_67.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_68.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_69.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_7.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_70.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_71.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_72.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_73.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_74.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_75.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_76.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_77.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_78.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_79.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_8.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_80.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_81.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_82.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_83.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_84.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_85.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_86.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_87.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_88.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_89.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_9.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_90.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_91.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_92.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_93.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_94.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_95.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_96.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_97.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_98.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_99.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_40.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_40.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_40.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_40.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_41.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_41.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_41.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_41.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_42.jpg
Processing image: gs://cuhk_fashion_report

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_1.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_10.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_100.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_101.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_102.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_103.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_104.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_105.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_106.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_107.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_108.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_109.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_11.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_110.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_111.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_112.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_113.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_114.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_115.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_116.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_117.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_118.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_119.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_12.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_120.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_121.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_122.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_123.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_124.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_125.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_126.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_13.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_14.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_15.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_16.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_17.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_18.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_19.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_2.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_20.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_21.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_22.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_23.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_24.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_25.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_26.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_27.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_28.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_29.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_3.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_30.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_31.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_32.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_33.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_34.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_35.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_36.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_37.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_38.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_39.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_4.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_40.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_41.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_42.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_43.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_44.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_45.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_46.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_47.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_48.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_49.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_5.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_50.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_51.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_52.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_53.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_54.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_55.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_56.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_57.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_58.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_59.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_1.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_1.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_1.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_1.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_10.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_10.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_10.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_10.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_100.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_6.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_60.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_61.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_62.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_63.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_64.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_65.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_66.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_67.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_68.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_69.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_7.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_70.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_71.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_72.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_73.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_74.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_75.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_76.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_77.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_78.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_79.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_8.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_80.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_81.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_82.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_83.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_84.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_85.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_86.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_87.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_88.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_89.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_9.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_90.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_91.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_92.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_93.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_94.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_95.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_96.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_97.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_98.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_99.jpg",
]

# Call the main function
process_images(image_paths, 4096)


Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_6.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_6.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_6.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_6.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_60.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_60.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_60.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_60.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_61.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_6

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN010JXLFt2Fp54Jfrmdy_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN011FmxwR2Fp54pX8xvl_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN011GRQz52Fp54iHUZ3e_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN011hO8fK2Fp52onIOg2_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN011Kgv9m2Fp4ylP3InF_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN011uGmBi2Fp4xo2mWDc_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN012N2VFu2Fp51JVABpl_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN012zEBtG2Fp53Q1b9bV_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN013OaLav2Fp55ERjy8o_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN013WOauS2Fp51Tx8iBG_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN014gQPrv2Fp55WZl9c7_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN014nKnQy2Fp54GaNytV_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN014zFcCX2Fp52EqsCe6_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0159Fqhc2Fp53jCqCRK_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0159tu702Fp528pEkGH_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN015MC4IX2Fp54ZeTVXA_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN015vTgAw2Fp54NzDCtb_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0168ucvv2Fp51B2Yj64_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN016pfhQ92Fp54YUlrKh_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN016X6yo52Fp52xopNwT_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN017AVUfg2Fp54vdXfXW_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0181Fv8w2Fp55Q2wXNt_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0187TikV2Fp54lAmMdZ_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN018EfwAK2Fp51BfyG9u_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN018GTG1W2Fp55DVnt8c_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN018lS1Ta2Fp55UHCMve_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN018qQxEw2Fp54ZeTNDj_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0198WmKE2Fp535z5o22_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN019kquza2Fp51OMaCUA_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01a0QeHE2Fp4wRZrqDW_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01AdZRdd2Fp53hEgFYF_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01af9kHi2Fp50D6zDH2_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01anfned2Fp54QZOluz_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01AR9Ee82Fp50v2xFqg_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01bB6alk2Fp53S2pMGw_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01BB6yla2Fp4ykJ4jOo_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01bDLKOF2Fp4wE0ogo2_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01beYYOC2Fp54L4pPuv_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01bhYNkL2Fp54uLKTA0_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01biuXGO2Fp4ypA51cL_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01BWwg1v2Fp5099a08M_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01cismqu2Fp53a5rUut_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01CjIqaG2Fp4xrXNxDk_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Cr7Glg2Fp4xngSbjC_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01CUb1o02Fp55ES0tE8_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01CVRFW42Fp509uAqg7_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01cYvIRx2Fp51pR23jO_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01d32A7M2Fp4wPXNoIs_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01DaIwia2Fp537xgBz4_!!0-item_pic.jpg",

]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN010JXLFt2Fp54Jfrmdy_!!2214702068928.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN010JXLFt2Fp54Jfrmdy_!!2214702068928.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN010JXLFt2Fp54Jfrmdy_!!2214702068928.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN010JXLFt2Fp54Jfrmdy_!!2214702068928.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN011FmxwR2Fp54pX8xvl_!!2214702068928.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN011FmxwR2Fp54pX8xvl_!!2214702068928.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN011FmxwR2Fp54pX8xvl_!!2214702068928.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN011FmxwR2Fp54p

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01DgmVAQ2Fp4xjt5vDx_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01DoOIsq2Fp51DApDkr_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01dpsaD22Fp52xfN6Qp_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01DQxA7a22qpWaSkEm8_!!0-tbbala.jpg_960x960.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01dWSnNQ2Fp55D9HYK3_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01dXSZSG2Fp54w7Nd0Z_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Dzpv0u2Fp52IZ7DI9_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01eaDtFs2Fp4xILTux2_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01eG2K9g2Fp4wrYIgOn_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01emuAQ52Fp54hJGXb6_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01eN1iOX2Fp52YzVNGV_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01f556iN2Fp54nqVxL7_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01fFe4L11Ka1sYf1kn0_!!0-tbbala.jpg_960x960.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Fl4Y1S2Fp55FVTZnV_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01fNMMi52Fp54VhKqoa_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01G0ow742Fp54IdUEdH_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01G8dygC2Fp4tOk5lEH_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01GCDzWv2Fp54mnt5ss_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01GXiDlF2Fp4xnBebx0_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01h2yL3f2Fp51kp4hsd_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01h3wpqE2Fp54C3anMU_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01h65dc72Fp54ujeW7E_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01HF1qFs2Fp5248OIQN_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01hHNLOW2Fp53l2R9x2_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01hJte4j2Fp51MHbU2k_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01hKLEiZ2Fp54S0WX54_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Hn0J232Fp53gycqc6_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01HnxIa22Fp5196dx3X_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01i4y0If2Fp4wPOrrST_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01IHkhXH2Fp54knzI4T_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01iKysRz2Fp51RWx2oj_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01iWmB552Fp55NxsqG7_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01J8682Z2Fp54eLppwZ_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01jbeTgm2Fp54fINnS7_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01jMXSnK2Fp50CsIgwK_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01jn3eOZ2Fp54nmeYTD_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Jrf0Lf2Fp508DNWS3_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01KEiNvm2Fp525zI7Bt_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01kic3GS2Fp52xWM3VG_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01KpCXMa2Fp54khJXfZ_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01kV8BCE2Fp55TJ4XHo_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01kyF8FR2Fp54H2ffhZ_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01LGamhI2Fp556SrMne_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01ljvxxa2Fp51dr7hp5_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01mmtOUm2Fp53sW0iYs_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01MtIido2Fp54z7IzWp_!!2214702068928.jpg",

]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01DgmVAQ2Fp4xjt5vDx_!!2214702068928.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01DgmVAQ2Fp4xjt5vDx_!!2214702068928.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN01DgmVAQ2Fp4xjt5vDx_!!2214702068928.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN01DgmVAQ2Fp4xjt5vDx_!!2214702068928.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01DoOIsq2Fp51DApDkr_!!2214702068928.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01DoOIsq2Fp51DApDkr_!!2214702068928.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN01DoOIsq2Fp51DApDkr_!!2214702068928.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN01DoOIsq2Fp51D

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01MTtWZW2Fp50q6VaIa_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01nbOoCG2Fp4xM4I9lC_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01NDJX3t2Fp4tIycXmP_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01NJscNj2Fp54H2G9sZ_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Nk8bAN2Fp54FgHGZP_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01NoH29h2Fp54ouINsF_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01nxEsPR2Fp52aT0VlH_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01o5Df8V2Fp52yAI6tY_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01OTXxIN2Fp53rgiLxe_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01P3hLiB2Fp52Lenu8L_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01pERj1z2Fp53MxsEWN_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Pjw1VM2Fp54GvcIkv_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01q5zSpq2Fp556oK79o_!!4611686018427385024-0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01qcW9EK2Fp51Qsl0rm_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01qQMVZD2Fp54FNBPrx_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01R8Pbjh2Fp53w3nrKW_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01rwNv5U2Fp55DLNUWT_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01s4T6JA2Fp53XejfeV_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01S92ZTF2Fp54HmnOsP_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01SpklBI2Fp55VMEzgh_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01tFmS9S2Fp4xQB0ZNx_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01TgV0aJ2Fp55GdvHcN_!!4611686018427385024-0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Tovphb2Fp4yztLTQC_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01TQUrFe2Fp54EQ1ayU_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01TxFf5R2Fp54Fzs8Gl_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01tzG5kB2Fp54w7xHBH_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01UAwjEp2Fp54nr6TMp_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01UdDAtD2Fp4yNReumF_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Ufel4s2Fp53qMfMKb_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01uiUlQe2Fp52Y7UNB9_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01UV79ET2Fp524vftyK_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01uXZrE92Fp52qIJi29_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01VhLY1h2Fp4xo2TFLj_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01vNACDW2Fp50iC9PuS_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01VpXeTO2Fp54oaWGEz_!!2214702068928.jpg",

]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01MTtWZW2Fp50q6VaIa_!!2214702068928.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01MTtWZW2Fp50q6VaIa_!!2214702068928.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN01MTtWZW2Fp50q6VaIa_!!2214702068928.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN01MTtWZW2Fp50q6VaIa_!!2214702068928.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01nbOoCG2Fp4xM4I9lC_!!2214702068928.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01nbOoCG2Fp4xM4I9lC_!!2214702068928.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN01nbOoCG2Fp4xM4I9lC_!!2214702068928.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN01nbOoCG2Fp4xM

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01VQIktR2Fp53b52XS0_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01vTZT4G2Fp4wUvZeMc_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01vveL542Fp54mofooF_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01W6n3nG2Fp52JeHkGM_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01wen15I2Fp55Fe7uzo_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01wHEZCQ2Fp51yATjUL_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01wiViQT2Fp4xbIhejb_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01wJHBas2Fp4xf2OMVG_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01WkIMAv25M5rDOiwzR_!!4611686018427383703-0-tbbala.jpg_960x960.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01wsuVHb2Fp55EoOQRG_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Xblmgx2Fp4ycWx6rI_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01XdoOaN2Fp55QbMXeJ_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01xpnZtb2Fp54hhrFuf_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01XQsNTp2Fp54GAE5JO_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01xZMXqi2Fp53NH4uxW_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01y3Cxdv2Fp4vyQq429_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01y5HvkV2Fp55EWMRHY_!!4611686018427385024-0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01ygiEPy2Fp54mWuka8_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Z9jiaq2Fp4xAvHIWP_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01zDhGRV2Fp4xixBVCP_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01zuW21b2Fp4wFGMGA9_!!2214702068928.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN010JsqCx1NTZ4whuVV8_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0115kDKo1NTZB4caSjW_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0117JNW91NTZ8tFWuCg_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN011kt7nQ1NTZBlLivEv_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN011wdkLw1NTZ3ppePuz_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN012Esh0A1NTZ8fXRSBV_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN012JKjqP1NTYzRil4hZ_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN012RlpQd1NTZ72VeDoP_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN013uQaLu1NTZ8kSNgw1_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN014adoR81NTZBpuEBUf_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN014BbTMe1NTZAtIdGtY_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN015aCF6e1NTZ999AF5C_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN016AEtLQ1NTZBxjkQVh_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN016F9oR81NTZATVsaO5_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN016hgLLK1NTZCPlEpBc_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN016HusY41NTZ42Q7mng_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN016lur521NTZ3XzHPuM_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0170SouS1NTZBenOBaZ_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0179XMOt1NTZ8uwUsKi_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN017Qg8HI1NTZ9DiiopF_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0183Gxf91NTZ2UmPHCr_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN018BdpoP1NTZB1NRHuP_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN018T8GN01NTZ8w5SXAM_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN0190fMCx1NTZBkWmd9W_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN019f4fzU1NTZBmAovch_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN019yAved1NTZ9I22LYk_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01A1i87F1NTZ8FJ0YKC_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01a9pzoU1NTZBgscMaI_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01ABckye1NTZ8qlnyZ6_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01AgO7I01NTZBsEpb5F_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01alxYG51NTZBolp1nr_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01amQPXW1NTZ6vkVjt7_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01AU7Rr31NTZBqqC66v_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01aXwUWp1NTZCODwsqp_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01b9Ztek1NTZC1ox78Y_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01BlxMFK1NTZ9mqrIBe_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01cCfPNF1NTZBnuoORI_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01cf2Cu11NTZD732Upo_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01CGT4tf1NTZ5EHwZq8_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01CKcyq21NTZ9EoHVCI_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01COUA3Y1NTZCM2v1B9_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01cqH5eS1NTZ6PohVlS_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01dA48mx1NTZ4syyp1q_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01DeDleZ1NTZB8wB6bm_!!2214496021571.png",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01dh97Fo1NTZ97h1PUV_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01DQ75SU1NTZ8q30tM6_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01DVznSw1NTZ9PZMXf7_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01dVZRMa1NTZ2T3BwqB_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01e04tpP1NTZ8TNA4vP_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01E4ofU91NTZ8fbhQHN_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01eLhuwf1NTZ4wk00UN_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01eneF8Z1NTZ2Nr6o37_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01eS5FDv1NTZ95qJENY_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01FuUKqN1NTZ60JjsEY_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01FUv7L91NTZ9IsgpeM_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01fwRPm61NTZC3YFEQi_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Gbbn331NTZ6zOO42F_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01gp8Hve1NTZ5NsDJzl_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01H44pXG1NTZBrfbumh_!!2214496021571.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01VQIktR2Fp53b52XS0_!!2214702068928.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01VQIktR2Fp53b52XS0_!!2214702068928.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN01VQIktR2Fp53b52XS0_!!2214702068928.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN01VQIktR2Fp53b52XS0_!!2214702068928.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01vTZT4G2Fp4wUvZeMc_!!2214702068928.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01vTZT4G2Fp4wUvZeMc_!!2214702068928.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN01vTZT4G2Fp4wUvZeMc_!!2214702068928.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN01vTZT4G2Fp4wU

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01hijwtQ1NTZB0uQ0rY_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01HYDtVD1NTZ58jUzuS_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01hzP7Hh1NTZ79xUyNF_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01ihJkT71NTZBnx0bqW_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Jejgs91NTZ7s14SkR_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01JgA2UF1NTZ2XAiTZj_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01jOk1hT1NTZ5H8uu5M_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01jPRFH91NTZCSUGf2I_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01kJbMU31NTZCTopbQ8_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01KkJUA31NTZ8vAbyc2_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01kQ2bzF1NTZB8moPtm_!!2214496021571.png",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01KSQ03L1NTZBrgsviA_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01KV1Qu61NTZ8zMLqde_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01lKRhyQ1NTZ979sO61_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01lPOuOZ1NTZ5KUKpLM_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01LupBtp1NTZ1J0GnRa_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01M4OyLS1NTZ5JMeKiS_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01m5nCES1NTZ2Mui7EV_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01M8iUZj1NTZCRagukE_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01MAbyPL1NTYzpIMPxu_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01MBxyVU1NTZ8Ko7joJ_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01mCNa9m1NTZ0CylFwY_!!2214496021571.jpg",

]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01hijwtQ1NTZB0uQ0rY_!!2214496021571.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01hijwtQ1NTZB0uQ0rY_!!2214496021571.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN01hijwtQ1NTZB0uQ0rY_!!2214496021571.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN01hijwtQ1NTZB0uQ0rY_!!2214496021571.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01HYDtVD1NTZ58jUzuS_!!2214496021571.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01HYDtVD1NTZ58jUzuS_!!2214496021571.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN01HYDtVD1NTZ58jUzuS_!!2214496021571.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN01HYDtVD1NTZ58

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01MlmNuI1NTZ52wYkf7_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01mPqpmP1NTZCaFAhsI_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01n3sg8Y1NTZB0dC2FN_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01NHXuOK1NTZ98A1SaT_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01nSlFm81NTZCMl9OXG_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01NXBDsr1NTZBCML76J_!!2214496021571.png",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01OCmIlI1NTZBczVf2v_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01op0lNn1NTZ7uK3hl7_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01oqYM7p1NTZ97TpQVZ_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01oz4Ya91NTZCRgKUEf_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01P4A4xn1NTZBgqJV8s_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01p4Qo2a1NTZ8VihMmq_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01Qco3oC1NTZ51m4EYa_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01QKkjLg1NTZBnwqCZq_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01QOMcl01NTZ6vPgwoi_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01qPT0Cc1NTZ8xpMkp7_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01r7JuZX1NTZ1cyTOFB_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01R8IsTa1NTZBAqkTdq_!!2214496021571.png",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01rAoVTW1NTZBpDfykR_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01RfLjDJ1NTZ9EJK9jW_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01rLloXr1NTZ8ivIzBV_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01rWBo4v1NTZBlPRSl4_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01sGOSS81NTZ70RxJwQ_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01sJDw9T1NTZ3sirme7_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01SKmjAp1NTZ5T5VjDU_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01SU3eyQ1NTZ2Dhg5Dm_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01TA3OiU1NTZ8rKYqMV_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01TidLor1NTZ4sJFr19_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01tjmP8D1NTZ3BxhWpS_!!2214496021571.jpg_q30.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01U2tvTP1NTZBpSHiAX_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01u98Li21NTZ8lKLzUk_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01ufHjBW1NTZ8K5d73N_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01V5FznG1NTZBunrVZ6_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01v8Pwdf1NTZC9EREhg_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01VbgEtX1NTZBwCh5iH_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01VImsEu1NTZBcYZxO3_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01vYJxQ81NTZCTDxCg3_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01W2x1OC1NTZB3dLSso_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01WLkQk41NTZ9BfRjn0_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01wm2Ck81NTZ9AE2FB2_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01WVRsKc1NTZ5Ufcdzh_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01x8IgEP1NTZ51Mej6n_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01XJcAtY1NTZENBIPff_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01XuDOIs1NTZ8Ko9Pja_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01xYKYsG1NTZCAzkFdu_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01yM05hI1NTYzYiQV21_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01YmwvVY1NTZBdscGfQ_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01YYrSVI1NTZ4xtxzrN_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01yyXoLa1NTZ8tfsFlm_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01ZCnXcN1NTZBolcwTj_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01zMNMGT1NTZ4sLEVSo_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01zpwEgq1NTZ8aOFhJD_!!0-item_pic.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01ZToFQq1NTZBmXO5H7_!!2214496021571.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/O1CN01zzhUrW1NTZCVIuV2j_!!2214496021571.jpg",

]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01MlmNuI1NTZ52wYkf7_!!2214496021571.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01MlmNuI1NTZ52wYkf7_!!2214496021571.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN01MlmNuI1NTZ52wYkf7_!!2214496021571.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN01MlmNuI1NTZ52wYkf7_!!2214496021571.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01mPqpmP1NTZCaFAhsI_!!2214496021571.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/O1CN01mPqpmP1NTZCaFAhsI_!!2214496021571.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/O1CN01mPqpmP1NTZCaFAhsI_!!2214496021571.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/O1CN01mPqpmP1NTZCa

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_103.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_110.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_118.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_119.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_12.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_126.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_135.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_145.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_148.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_15.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_18.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_26.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_32.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_43.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_46.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_55.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_61.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_63.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_64.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_69.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_72.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_73.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_103.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_110.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_118.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_119.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_12.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_126.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_135.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_145.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_148.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_15.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_18.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_26.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_32.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_43.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_46.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_55.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_61.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_63.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_64.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_69.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_72.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_2_Dazzle_73.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_103.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_110.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_118.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_119.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_12.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_126.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_135.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_145.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_148.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_15.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_18.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_26.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_32.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_43.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_46.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_55.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_61.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_63.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_64.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_69.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_72.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/2_Dazzle_73.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_103.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/2_2_2_Dazzle_103.jpg (Attempt 1/3)


In [ ]:
#New Image

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/16636943_55077891_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/18111187_53241225_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/18940396_41254355_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/19039015_41615023_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/20071315_45236365_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/20071327_45238521_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/20137257_54812302_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/20567645_51046349_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21031189_51090968_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21293148_51421015_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21294031_51382611_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21294043_51365569_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21477188_51445632_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21477224_51476271_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21514716_53148812_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21614206_51827568_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21614207_51797437_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21670084_51993455_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21670087_51993534_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/21928679_51708309_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/23780196_53778876_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/23780277_53778814_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/23867763_53942781_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/23926257_54070138_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24136632_54238570_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24136852_54240149_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24136866_54210057_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24137253_54245199_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24137326_54240492_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24137478_54239203_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24137589_54231790_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24137592_54239320_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24179902_54287290_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24179904_54286992_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24179912_54286565_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24287367_54365844_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24287368_54369270_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24287369_54368946_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24287372_54368604_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24287374_54357370_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24287375_54357453_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24313246_54428838_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24608888_54948398_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24609406_54947554_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24609411_54947637_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24609415_54947507_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/24609418_54949330_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25099119_55320246_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25099161_55325616_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25099234_55320028_480.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/16636943_55077891_480.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/16636943_55077891_480.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/16636943_55077891_480.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/16636943_55077891_480.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/18111187_53241225_480.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/18111187_53241225_480.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/18111187_53241225_480.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/18111187_53241225_480.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/18940396_41254355_480.j

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_146.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_147.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_148.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_149.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_15.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_150.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_16.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_17.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_19.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_2.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_20.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_21.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_22.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_23.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_24.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_25.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_26.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_27.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_28.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_29.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_3.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_30.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_31.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_32.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_33.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_34.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_35.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_36.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_37.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_39.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_4.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_40.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_41.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_42.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_43.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_44.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_45.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_46.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_48.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_49.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_5.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_50.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_51.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_52.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_53.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_54.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_55.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_56.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_57.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/Edition_58.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_146.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_146.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Edition_146.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Edition_146.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_147.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_147.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Edition_147.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Edition_147.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_148.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_148.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/25099246_55320050_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25099353_55325584_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25099356_55320099_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25099364_55320068_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25099378_55326225_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25099390_55325567_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25099401_55319977_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25099442_55326160_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25203463_55501513_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25203474_55501548_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25203486_55501699_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25232817_55714144_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25233021_55714187_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25369619_55811924_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25630657_56095025_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25631270_56094855_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25631276_56094828_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25631277_56093031_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25702743_56058406_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25827872_56242409_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25827886_56067051_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25827923_56066268_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25827925_56067157_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25827927_56066089_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25867831_56067099_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25867912_56067458_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25867928_56068027_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25871262_55917733_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25878436_56067096_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/25893045_56066294_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/26339998_56214544_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/26343061_56214440_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/26366532_56029497_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/27056633_56848769_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/27056666_56849189_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/27057367_56848920_480.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/AHU464_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ALE471_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/AOU011_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/AQG846_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/BOW726_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/BQH424_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/BQH425_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/BUB301_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/BUL196_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/BVR618_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/BVY718_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_1.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_10.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_102.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/25099246_55320050_480.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/25099246_55320050_480.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/25099246_55320050_480.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/25099246_55320050_480.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/25099353_55325584_480.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/25099353_55325584_480.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/25099353_55325584_480.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/25099353_55325584_480.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/25099356_55320099_480.j

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/Edition_103.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_104.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_105.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_106.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_107.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_108.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_109.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_11.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_110.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_111.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_112.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_113.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_114.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_115.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_116.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_117.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_118.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_119.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_12.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_120.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_121.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_122.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_123.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_124.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_125.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_126.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_127.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_128.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_129.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/BWB070_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_130.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_131.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_132.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_133.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_134.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_135.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_136.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_137.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_138.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_139.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_14.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_140.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_141.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_142.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_143.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_144.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_145.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_146.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_147.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_148.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_103.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_103.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Edition_103.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Edition_103.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_104.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_104.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Edition_104.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Edition_104.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_105.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_105.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_15.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_150.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_16.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_17.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_18.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_19.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_2.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_20.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_21.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_22.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_23.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_24.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_25.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_26.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_27.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_28.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_29.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_3.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_30.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_31.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_32.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_33.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_34.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_35.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_36.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_37.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_38.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_39.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_4.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_40.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_41.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_42.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_43.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_44.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_45.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_46.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_47.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_48.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_49.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_5.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_50.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_51.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_52.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_53.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_54.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_55.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_56.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_57.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_58.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_59.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_15.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_15.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_15.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_15.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_150.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_150.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_150.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_150.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_16.jpg
Processing image: gs://cuhk_fashion_report_2024

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/Edition_149.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_15.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_150.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_16.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_17.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/BWL515_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_19.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_2.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_20.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_21.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_22.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_23.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_24.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_25.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_26.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_27.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_28.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_29.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_3.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_30.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_31.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_54.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_83.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_106.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_126.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_135.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ELLASSAY_歌力思_22.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EAE908_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EAG990_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EAJ540_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EAK958_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EAK969_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EAO459_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EAQ531_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBB769_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBH970_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBH976_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBH980_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBI519_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBJ065_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBJ479_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBO466_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBU255_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBU901_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBU905_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBV801_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBX002_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBX008_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBX643_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBX659_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_149.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_149.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Edition_149.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Edition_149.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_15.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_15.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/Edition_15.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/Edition_15.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_150.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_150.jpg (Attempt 1/3)


In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EBY080_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EBY738_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_100.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_13.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_18.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_38.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_47.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Edition_7.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEA853_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEA865_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEA884_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEE062_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEE366_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEE374_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEE719_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEG774_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEI804_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEI809_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEI824_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEI886_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEI898_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEI959_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEI989_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEJ442_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEJ567_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEJ644_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEJ707_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEJ920_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEK245_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEK247_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEK579_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEL287_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEL462_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEL683_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEL756_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEL883_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEU110_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEW067_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEX387_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EEX549_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGE303_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGE715_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGI278_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGJ593_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGK515_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGK825_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGL069_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGL075_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGL077_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGL086_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EBY080_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EBY080_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EBY080_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EBY080_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EBY738_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EBY738_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EBY738_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EBY738_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_100.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/Edition_100.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EGO643_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGO781_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGR058_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGR072_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGR121_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGR137_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGR739_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGR743_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGU434_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGU921_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGU925_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGU929_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGU979_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV068_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV120_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV122_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV124_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV128_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV133_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV135_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV137_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV147_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV148_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV367_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV864_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV963_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGV986_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGW977_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGX435_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGY492_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGY570_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGY796_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGY801_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGY802_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EGY822_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA362_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA365_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA371_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA375_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA380_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA451_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA457_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA472_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA477_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA482_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA483_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA494_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA496_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA515_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA519_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EGO643_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EGO643_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EGO643_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EGO643_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EGO781_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EGO781_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EGO781_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EGO781_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EGR058_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EGR058_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EHA520_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA530_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA683_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA690_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA697_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA814_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA825_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHA889_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB024_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB286_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB287_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB289_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB293_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB360_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB367_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB373_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB445_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB449_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB832_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB842_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB850_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB852_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB867_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB888_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB901_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB905_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB978_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB980_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB988_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHB993_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHE219_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHE222_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHE225_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHE798_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHG035_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHG108_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHG224_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHG579_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHG580_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHG739_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHG846_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHG949_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHH482_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHH836_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHH849_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHI085_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHI998_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHJ051_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHJ905_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHK232_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHA520_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHA520_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EHA520_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EHA520_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHA530_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHA530_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EHA530_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EHA530_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHA683_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHA683_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EHK240_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHK244_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHK247_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHK262_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHK267_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHK270_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHK278_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHK434_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHL290_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHO390_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHQ019_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHR048_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHR493_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHR508_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHR514_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHR545_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHR643_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHR645_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHR649_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHR901_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHR903_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHU755_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHV399_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHV402_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHV434_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHV435_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHV438_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHV443_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHV445_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHV448_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHV630_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHW564_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHW913_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHW960_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX325_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX553_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX562_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX592_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX727_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX748_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX773_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX784_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX860_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX863_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX869_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX871_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX878_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX886_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX949_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHX952_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHK240_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHK240_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EHK240_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EHK240_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHK244_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHK244_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EHK244_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EHK244_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHK247_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHK247_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EHY082_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY161_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY166_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY244_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY257_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY298_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY299_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY307_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY380_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY383_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY385_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY408_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY458_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EHY614_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA177_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA233_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA253_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA254_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA285_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA286_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA292_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA296_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA335_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA338_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA520_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA529_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA591_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA616_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA617_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA639_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA726_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA728_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA731_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA745_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA753_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA765_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA774_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA775_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA777_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA782_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA786_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA855_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA858_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA936_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA938_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIA962_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB049_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB053_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB054_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB065_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHY082_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHY082_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EHY082_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EHY082_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHY161_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHY161_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EHY161_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EHY161_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHY166_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EHY166_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIB088_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB115_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB121_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB148_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB150_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB152_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB447_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB752_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB872_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB875_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB877_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB880_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB889_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB892_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB895_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB902_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB922_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB924_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB938_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB942_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB962_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB963_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB968_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB974_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIB983_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE005_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE008_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE011_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE023_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE028_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE062_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE069_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE094_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE096_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE116_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE122_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE125_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE138_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE139_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE142_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE171_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE180_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE186_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE216_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE298_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE504_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE506_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE593_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE601_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE618_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIB088_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIB088_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIB088_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIB088_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIB115_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIB115_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIB115_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIB115_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIB121_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIB121_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIE631_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE651_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE658_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE674_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE676_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE685_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE697_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE711_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE720_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE753_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE780_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE783_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE784_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE785_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE906_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE910_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE928_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE930_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE931_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE934_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE936_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE941_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE943_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE946_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE962_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIE963_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG147_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG150_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG165_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG234_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG306_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG309_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG315_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG329_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG335_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG340_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG344_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG354_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG357_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG362_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG364_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG481_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG483_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG510_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG537_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG563_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG575_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG622_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG777_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG832_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIE631_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIE631_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIE631_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIE631_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIE651_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIE651_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIE651_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIE651_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIE658_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIE658_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIG833_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG838_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG845_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG847_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG850_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG851_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG852_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG862_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG863_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG868_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG876_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG892_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG898_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG904_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG906_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG908_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG911_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG912_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG917_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG918_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG938_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG942_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG945_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIG946_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH039_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH059_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH068_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH076_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH111_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH159_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH166_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH168_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH169_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH170_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH172_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH192_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH196_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH201_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH204_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH252_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH253_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH254_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH264_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH296_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH299_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH300_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH305_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH728_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH730_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIH942_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIG833_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIG833_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIG833_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIG833_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIG838_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIG838_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIG838_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIG838_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIG845_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIG845_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIH982_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII083_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII092_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII105_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII116_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII121_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII152_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII178_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII185_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII421_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII431_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII439_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII480_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII500_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII505_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII543_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII546_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII549_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII552_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII560_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII572_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII590_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII601_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII607_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII776_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII786_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII788_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII789_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII799_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII800_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII821_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII830_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII831_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII850_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII854_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII864_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EII871_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ007_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ055_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ061_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ072_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ079_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ105_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ134_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ148_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ190_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ209_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ210_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ211_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ251_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIH982_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIH982_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIH982_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIH982_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EII083_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EII083_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EII083_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EII083_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EII092_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EII092_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIJ256_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ260_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ263_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ386_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ393_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ397_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ399_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ406_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ461_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ479_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ481_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ539_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ551_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ560_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ562_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ583_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ584_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ594_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ597_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ600_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ608_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ626_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ630_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ636_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ646_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ648_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ650_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ653_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ655_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ669_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ674_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ687_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ694_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ717_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ721_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ736_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ741_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ746_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ806_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ826_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ829_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ834_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ850_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ907_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ920_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ954_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIJ972_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK065_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK115_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK124_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIJ256_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIJ256_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIJ256_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIJ256_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIJ260_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIJ260_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIJ260_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIJ260_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIJ263_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIJ263_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIK133_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK135_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK137_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK154_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK160_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK162_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK164_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK172_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK175_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK202_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK203_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK212_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK222_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK225_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK250_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK251_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK406_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK412_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK430_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK436_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK441_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK444_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK504_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK505_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK506_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK514_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK517_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK518_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK544_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK548_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK553_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK589_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK590_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK615_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK627_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK668_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK674_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK913_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK916_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK940_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK946_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK949_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK955_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK963_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK966_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK968_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK972_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK976_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIK979_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIL027_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIK133_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIK133_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIK133_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIK133_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIK135_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIK135_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIK135_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIK135_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIK137_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIK137_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO033_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO047_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO054_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO057_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO071_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO102_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO140_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO205_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO229_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO232_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO242_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO244_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO607_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO613_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO626_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO650_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO656_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO667_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO671_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO677_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO834_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO836_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO876_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO910_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIO989_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ012_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ016_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ022_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ025_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ029_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ033_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ040_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ054_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ096_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ110_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ131_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ161_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ164_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ169_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ171_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ290_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ302_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ303_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ330_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ343_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ369_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ387_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIQ393_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO033_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO033_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

In [ ]:
# Main function to process multiple images with retries
def process_images(image_paths, max_size):
    for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

            # Example usage for multiple images
            image_paths = [
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ405_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ407_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ441_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ476_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ477_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ479_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ490_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ523_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ535_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ548_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ549_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ578_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ605_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ618_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ627_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ631_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ740_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ836_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ837_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ856_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ876_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ892_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ897_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ901_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ906_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ915_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIQ922_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR166_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR201_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR205_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR225_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR252_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR307_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR311_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR330_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR337_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR339_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR340_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR342_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR375_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR380_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR425_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR452_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR458_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR468_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR473_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR487_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR616_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR637_in_l.jpg",
            "gs://cuhk_fashion_report_2024_image_batch1/EIR641_in_l.jpg",
            ]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO033_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO033_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
      for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

        # Example usage for multiple images
        image_paths = [
        "gs://cuhk_fashion_report_2024_image_batch1/EIR647_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR650_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR654_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR658_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR660_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR665_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR675_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR678_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR834_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR845_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR848_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR851_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR857_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR970_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR981_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR986_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIR994_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU029_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU030_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU034_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU040_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU046_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU059_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU064_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU073_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU080_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU142_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU150_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU174_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU194_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU291_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU313_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU321_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU405_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU408_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU452_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU472_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU473_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU495_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU497_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU595_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU603_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU604_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU613_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU618_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU639_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU650_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU670_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU757_in_l.jpg",
        "gs://cuhk_fashion_report_2024_image_batch1/EIU760_in_l.jpg",
        ]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIL665_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIO019_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO033_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIO033_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
          "gs://cuhk_fashion_report_2024_image_batch1/EIU761_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU768_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU777_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU780_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU789_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU794_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU798_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU800_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU909_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU910_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU911_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU912_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU936_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU942_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU951_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIU968_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV204_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV263_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV269_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV287_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV301_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV315_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV325_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV326_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV329_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV331_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV332_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV358_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV367_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV369_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV377_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV379_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV400_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV427_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV444_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV457_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV461_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV466_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV492_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV493_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV598_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV720_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV735_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV810_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV837_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV838_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV850_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV859_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV930_in_l.jpg",
          "gs://cuhk_fashion_report_2024_image_batch1/EIV939_in_l.jpg",
    ]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIU761_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIU761_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIU761_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIU761_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIU768_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIU768_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIU768_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIU768_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIU777_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIU777_in_l.jpg (Attempt 1

In [16]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIV943_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW053_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW057_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW058_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW118_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW124_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW126_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW129_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW137_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW141_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW152_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW154_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW155_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW159_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW184_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW188_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW196_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW204_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW205_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW207_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW221_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW259_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW261_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW265_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW269_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW279_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW302_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW315_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW322_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW338_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW354_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW356_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW361_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW368_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW369_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW439_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW445_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW460_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW483_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW484_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW496_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW498_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW500_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW512_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW514_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW526_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW544_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW553_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW567_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW602_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIV943_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIV943_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIV943_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIV943_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIW053_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIW053_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIW053_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIW053_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIW057_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIW057_in_l.jpg (Attempt 1

In [17]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIW720_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW725_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW751_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW754_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW798_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW808_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW816_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW817_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW957_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW965_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW969_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW970_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIW990_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX009_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX010_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX013_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX014_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX033_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX049_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX063_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX066_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX067_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX076_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX081_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX087_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX141_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX147_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX149_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX155_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX167_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX178_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX180_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX194_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX198_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX212_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX247_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX252_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX260_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX263_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX271_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX285_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX290_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX317_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX324_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX337_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX338_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX352_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX373_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX382_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX468_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIW720_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIW720_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIW720_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIW720_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIW725_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIW725_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIW725_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIW725_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIW751_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIW751_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIX470_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX473_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX480_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX569_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX575_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX580_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX584_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX587_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX593_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX613_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX614_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX615_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX632_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX643_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX660_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX814_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX816_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX824_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX825_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX826_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX902_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIX909_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY082_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY083_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY085_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY092_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY096_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY122_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY129_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY145_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY208_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY457_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY459_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY476_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY498_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY546_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY556_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY561_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY566_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY584_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY712_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY713_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY817_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY901_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY924_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY928_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY936_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY937_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY944_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY963_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIX470_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIX470_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIX470_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIX470_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIX473_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIX473_in_l.jpg (Attempt 1/3)
Result for image gs://cuhk_fashion_report_2024_image_batch1/EIX473_in_l.jpg appended to /content/drive/MyDrive/Deep_Learning_Project/fashion_data.xlsx
Image gs://cuhk_fashion_report_2024_image_batch1/EIX473_in_l.jpg successfully processed and saved.
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIX480_in_l.jpg
Processing image: gs://cuhk_fashion_report_2024_image_batch1/EIX480_in_l.jpg (Attempt 1

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EIY985_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY986_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY997_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY998_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EIY999_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA051_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA065_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA098_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA135_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA141_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA297_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA298_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA323_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA327_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA353_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA363_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA368_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA369_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA379_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA483_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA724_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA728_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA740_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA744_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA747_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA754_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA758_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA761_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA834_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA838_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA865_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA870_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA919_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA929_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA953_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA966_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA984_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJA999_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB150_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB169_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB189_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB236_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB243_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB267_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB282_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB293_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB301_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB312_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB326_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB348_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EJB350_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB448_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB454_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB456_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB465_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB470_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB492_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB493_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB494_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB499_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB537_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB576_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB592_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB601_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB606_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB615_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB616_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB618_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB651_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB653_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB661_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB709_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB711_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB712_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB728_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB732_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB737_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB742_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB753_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB765_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB772_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB801_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB826_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB864_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB880_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB883_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB892_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB904_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB917_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB920_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB939_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB940_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB942_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB946_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB947_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB952_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB970_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB971_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB976_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB978_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EJB983_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB986_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJB987_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE008_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE078_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE105_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE106_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE118_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE157_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE174_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE183_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE187_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE194_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE282_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE283_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE295_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE422_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE452_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE525_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE529_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE540_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE545_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE548_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE549_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE556_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE567_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE607_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE611_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE715_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE719_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE741_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE743_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE747_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE811_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE844_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE955_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE958_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJE963_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG108_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG423_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG436_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG473_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG476_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG501_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG515_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG531_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG533_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG581_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG586_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG597_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EJG888_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG921_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJG948_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH127_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH194_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH210_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH214_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH222_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH504_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH536_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH537_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH540_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH556_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH559_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH570_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH594_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH600_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH602_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH608_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH610_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH725_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH728_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH842_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH897_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH911_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH986_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJH993_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI152_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI154_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI156_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI160_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI170_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI172_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI173_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI174_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI219_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI222_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI265_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI275_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI281_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI284_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI289_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI295_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI377_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI416_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI429_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI453_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI464_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI475_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI487_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EJI497_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI612_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI625_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI632_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI653_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI728_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI729_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI736_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI746_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI770_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI797_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJI840_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ063_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ116_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ118_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ132_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ136_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ137_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ250_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ258_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ261_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ266_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ320_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ350_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ352_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ385_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ390_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ466_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJJ832_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK032_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK054_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK058_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK068_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK078_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK095_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK123_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK126_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK136_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK201_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK234_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK235_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK240_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK254_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK272_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK293_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK317_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK318_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK319_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK325_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK333_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EJK334_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK495_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK505_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK511_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK532_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK536_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK541_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK557_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK659_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK789_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK813_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK815_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK990_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK993_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK995_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK996_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJK997_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL001_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL015_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL018_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL043_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL064_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL076_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL077_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL079_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL086_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL092_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL094_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL100_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL101_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL102_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL106_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL121_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL138_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL246_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL266_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL281_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL317_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL478_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL505_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL669_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL671_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL679_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL735_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL910_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL916_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL922_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL926_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL944_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL951_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EJL954_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL966_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL974_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL976_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJL979_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO031_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO084_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO187_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO196_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO371_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO519_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO533_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO537_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO541_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO611_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO612_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO615_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO654_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO688_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO693_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO694_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO702_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO707_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO708_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO715_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJO722_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ018_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ022_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ025_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ047_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ055_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ170_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ271_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ282_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ288_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ308_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ321_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ340_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ361_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ371_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ411_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ416_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ445_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ449_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ558_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ562_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ566_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ567_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ696_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ697_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EJQ709_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJQ738_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR014_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR088_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR135_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR152_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR160_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR165_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR172_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR225_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR228_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR233_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR237_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR388_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR397_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR398_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR632_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR666_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR775_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR788_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR790_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR816_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR819_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR822_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJR872_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU067_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU069_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU449_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU463_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU481_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU606_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU654_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU656_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU674_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU819_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJU882_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV007_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV019_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV220_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV278_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV286_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV293_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV320_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV327_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV411_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV425_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV777_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV781_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV810_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV812_in_l.jpg",

]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EJV817_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV823_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV825_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV826_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV832_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV835_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV838_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV849_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJV850_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW014_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW018_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW023_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW026_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW116_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW172_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW228_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW370_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW483_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW489_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW496_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW498_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW501_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJW763_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX052_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX058_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX171_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX174_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX175_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX178_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX195_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX263_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX268_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX270_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX308_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX341_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX367_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX440_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX447_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX550_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX598_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX612_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX626_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX658_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX669_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX705_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX764_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJX968_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY055_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY062_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY076_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/EJY262_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY265_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY436_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY451_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY452_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY457_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY458_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY460_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY506_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY512_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY582_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY585_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY586_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EJY745_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKA005_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKA035_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKA179_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKA188_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKA190_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKA197_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKA204_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1113_15431c34-ec86-40b2-8234-4f192c871119_1170x.png",
"gs://cuhk_fashion_report_2024_image_batch1/pbm20240126j1114_5b3555bc-ea37-43e5-9069-fb11029f30b2_1880x.png",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_1.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_10.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_100.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_101.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_102.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_103.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_104.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_105.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_106.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_107.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_108.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_109.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_11.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_110.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_111.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_112.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_113.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_114.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_115.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_116.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_117.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_118.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_119.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_12.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_120.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_121.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_122.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_123.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_124.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_125.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_126.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_127.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_128.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_129.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_13.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_130.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_131.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_132.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_133.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_134.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_135.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_136.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_137.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_138.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_139.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_14.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_140.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_141.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_142.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_143.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_144.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_145.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_146.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_147.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_148.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_15.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_16.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_17.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_18.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_19.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_2.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_20.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_21.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_22.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_23.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_24.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_25.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_26.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_27.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_28.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_29.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_3.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_30.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_31.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_32.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_33.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_34.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_35.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_36.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_37.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_38.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_39.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_4.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_40.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_41.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_42.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_43.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_44.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_45.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_46.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_47.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_48.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_49.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_5.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_50.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/Peacebird_太平鳥_51.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKB595_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKB675_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_78.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_8.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_81.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKB868_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKB875_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKB897_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKB979_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_9.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_90.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_92.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_93.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKE850_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKG315_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKG323_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKG330_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN011FmxwR2Fp54pX8xvl_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN011GRQz52Fp54iHUZ3e_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01HF1qFs2Fp5248OIQN_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01hHNLOW2Fp53l2R9x2_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01hJte4j2Fp51MHbU2k_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01Hn0J232Fp53gycqc6_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01i4y0If2Fp4wPOrrST_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01iWmB552Fp55NxsqG7_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01jbeTgm2Fp54fINnS7_!!0-item_pic.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01jMXSnK2Fp50CsIgwK_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01jn3eOZ2Fp54nmeYTD_!!0-item_pic.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01Jrf0Lf2Fp508DNWS3_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01KEiNvm2Fp525zI7Bt_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01kV8BCE2Fp55TJ4XHo_!!2214702068928.jpg",
]

# Call the main function
process_images(image_paths, 4096)

In [ ]:
    credentials, _ = default()
    auth_request = transport.requests.Request()
    credentials.refresh(auth_request)

    MODEL_LOCATION = "us-central1"
    MAAS_ENDPOINT = f"{MODEL_LOCATION}-aiplatform.googleapis.com"

    client = openai.OpenAI(
        base_url=f"https://{MAAS_ENDPOINT}/v1beta1/projects/{PROJECT_ID}/locations/{LOCATION}/endpoints/openapi",
        api_key=credentials.token,
    )

# Main function to process multiple images with retries
def process_images(image_paths, max_size):
  for image_path in image_paths:
        try:
            # Ensure the image path is passed correctly and log for debugging
            print(f"Processing image: {image_path}")

            # Call process_images_with_retry with the correct input
            process_images_with_retry([image_path], max_size)  # Wrap image_path in a list if needed

        except Exception as e:
            # Catch any error, log it, and continue with the next image
            print(f"Error processing image {image_path}: {e}. Skipping to next image.")

# Example usage for multiple images
image_paths = [
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01ljvxxa2Fp51dr7hp5_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01mmtOUm2Fp53sW0iYs_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01MtIido2Fp54z7IzWp_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01MTtWZW2Fp50q6VaIa_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01nbOoCG2Fp4xM4I9lC_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01NDJX3t2Fp4tIycXmP_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01zuW21b2Fp4wFGMGA9_!!2214702068928.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN013uQaLu1NTZ8kSNgw1_!!2214496021571.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/O1CN01nSlFm81NTZCMl9OXG_!!2214496021571.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKH608_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/EKH612_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_79.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_80.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_83.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_85.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_86.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_87.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_88.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_89.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_91.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_94.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_95.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_96.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_97.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_98.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/ShushuTong_99.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/WOR104_in_l.jpg",
"gs://cuhk_fashion_report_2024_image_batch1/XXE300_in_l.jpg",
]

# Call the main function
process_images(image_paths, 4096)